# Fehlertoleranz der Spotify-Datenanalye

## Einführung
Fehlertoleranz ist ein zentraler Aspekt in verteilten Systemen wie Apache Spark. In diesem Notebook wird untersucht, wie robust diese Spark-Anwendung gegenüber Fehlern und Ausfällen ist.

**Dabei betrachten wir drei Hauptaspekte:**

- **1. Fehlertoleranz im Code**
    - Bereits in der Implementierung wurden Maßnahmen ergriffen, um Fehler oder problematische Daten frühzeitig abzufangen.
    - Beispielsweise wurden bestimmte Daten gefiltert oder Bereinigungen vorgenommen, um spätere Fehler zu vermeiden.
    - Zudem nutzen einige Transformationen und Mechanismen von Spark bereits integrierte Fehlertoleranzmechanismen, die automatisch für eine stabilere Verarbeitung sorgen können.

- **2. Fehlertoleranz-Test**
    - Um die Stabilität der Spark-Anwendung zu bewerten, werden gezielt Fehlerfälle simuliert und ihr Einfluss auf die Anwendung analysiert.
 
- **3. Weitere potentielle Fehlerszenarien** *(Am Ende des Notebooks)*

Das **Ziel dieses Notebooks** ist es, besser zu verstehen, wie gut die Anwendung mit Fehlern umgehen kann und welche potenziellen Schwachstellen existieren.

---

### 1. Fehlertoleranz beim Laden der Daten

- **beschädigte Dateien** werden durch das `try-except` in der Funktion `process_pickle()`**abgefangen**.
- Dabei werden **Standardwerte für fehlende Felder gesetzt**.
- **Fehlerhafte Dateien** werden mit `"track_uri": None` **markiert**.

In [ ]:
 try:
        with open(filepath, "rb") as file:
            pickle_data = pickle.load(file)
        return {
            "track_uri": pickle_data.get("track_uri", "Unbekannt"),
            "bars": pickle_data.get("bars", []),
            "duration_ms": pickle_data.get("duration_ms", "Unbekannt"),
            "sections": pickle_data.get("sections", []),
            "segments": pickle_data.get("segments", []),
            "loudness_max": pickle_data.get("loudness_max", "Unbekannt"),
            "keys": pickle_data.get("keys", []),
            "track": pickle_data.get("track", [])
        }
    except Exception as e:
        return {"track_uri": None, "error": str(e)}

### 2. Fehlertoleranz durch Debugging-Informationen  

Um sicherzustellen, dass bei der Kombination von **CSV- und Pickle-Dateien** keine oder nur wenige Daten verloren gehen, wird die **Anzahl der Zeilen vor und nach der** `Join`-**Operation** gezählt.  

Eine signifikante Abnahme der Zeilenanzahl könnte darauf hindeuten, dass die Werte in der Spalte `track_uri` nicht übereinstimmen. Da die **CSV-Daten bereits im initialen Schritt (Schritt 0)** sorgfältig bereinigt wurden, sollte dies normalerweise nicht auftreten.  

Falls dennoch größere Abweichungen zwischen den Zeilen vor und nach dem Join festgestellt werden, könnte dies ein Hinweis darauf sein, dass der Pfad `csc_dir` auf eine **noch unbereinigte Datei** verweist.
Des Weiteren wird in diesem Schritt durch die Zählung und anschließende Ausgabe der **erfolgreich sowie nicht verarbeiteten Dateien** für weitere Fehlertoleranz gesorgt.  

Die zusätzliche Ausgabe der ersten Zeilen des kombinierten DataFrames hilft außerdem dabei, **mögliche Fehler im DataFrame** schon **vor komplexen und teuren Datenverarbeitungen frühzeitig zu identifizieren**.


In [ ]:
# Join-Information fürs Debugging
print(f"JOIN Information:")
print(f"- Zeilen vor Join: {valid_pickle_df.count()}")
print(f"- Zeilen nach Join: {combined_df.count()}")

# Statistik anzeigen
total_files = len(pickle_files)
successful_files = valid_pickle_df.count()
failed_files = total_files - successful_files

print(f"")
print(f"Erfolgs-Statistik:")
print(f"- Gesamtanzahl der Dateien: {total_files}")
print(f"- Erfolgreich verarbeitet: {successful_files}")
print(f"- Fehlerhaft: {failed_files}")

# Test, ob CSV und Pickle korrekt gejoint wurden: 
combined_df.select("track_uri", "track_uri_csv", "name", "duration_ms").show(n=10, truncate=False)

### 3. Fehlertoleranz bei der Berechnung des Dur-Prozentsatzes

`F.when(combined_df["mode"] == 1, 1).otherwise(0)` stellt sicher, dass nur die Werte 0 und 1 aufgenommen werden. **Sollte die Mode-Spalte andere Werte haben**, wird **keine Fehlermeldung** ausgegeben, sondern der Wert eindeutig eingeordnet. 

In [10]:
combined_df = combined_df.withColumn("is_major", F.when(combined_df["mode"] == 1, 1).otherwise(0))

+----------------+
|major_percentage|
+----------------+
|            65.0|
+----------------+



### 4. Fehlertoleranz bei der Datenvorbereitung für die Korrelation 

Um **Zeilen mit** `Null`-**Werten zu entfernen** und eine zuverlässige Berechnung der Korrelationswerten zu gewährleisten, wird die Filterfunktion `filter(isNotNull())` angewendet. 


In [ ]:
filtered_df = exploded_df.filter((F.col("energy").isNotNull()) & (F.col("danceability").isNotNull()) & (F.col("loudness_max_segment").isNotNull()))

### Fehlertoleranz im Code - Fazit   

Der Code hat gezielte Maßnahmen zur Fehlertoleranz implementiert, sodass keine unerwarteten Abstürze oder falschen Ergebnisse vorkommen sollten.   

  - Explizite Datenbereinigung vor erstmaliger Ausführung
  - Fehlerbehandlung bei Pickle-Dateien mit defekten oder fehlenden Daten, sodass der Prozess nicht unterbrochen wird
  - Vermeidung von Null-Werten um Berechnungen zuverlässig durchzuführen
  - Fehlerminimierung bei kritischen Operationen wie `explode()` oder `join()`: Ergänzung mit geeigneten Maßnahmen um z.B. Datenverlust zu reduzieren.

---

## Fehlertoleranz-Tests

### Testumgebung

Da die lokale Ausführung der Spark-Anwendung nicht ausreichte, um realistische Fehlertoleranz-Szenarien zu testen, wurde stattdessen Spark im **Standalone-Modus** eingerichtet *(analog zur Anleitung aus dem Lab)*. 

<img src="../data/images/sparkcluster.png" alt="Alle Worker aktiv" width="600">

*Überblick der gestarteten Worker und Masters in der Spark-UI*

Die folgenden Hard- und Software-Spezifikationen wurden für die Fehlertoleranz-Tests verwendet:  

- **1 Master Node**
- **4 Worker Nodes**
    - **RAM**: 4 GB
    - **CPU-Kerne**: 2

<img src="../data/images/standaloneoverview.png" alt="Alle Worker aktiv" width="300">

*Ressourcen und Status des Clusters in der Master-UI*

Wenn Größen im Test angepasst, verändert oder ganz weggelassen wurden, wurde dies explizit angegeben.

<img src="../data/images/workeralive.png" alt="Alle Worker aktiv" width="800">

*Alle erstellten Worker und ihre Ressourcen auf einem Blick in der Master-UI*

## Zielsetzung
Der Test soll zeigen, wie sich die Spark-Anwendung verhält, wenn:  
1.  während einer Spark-Anwendung **Worker-Nodes ausfallen**
2.  das **Netzwerk ausfällt**
3.  **Worker-Nodes stark ungleichmäßig belastet werden**    

Im weiteren Verlauf dieses Notebooks werden die Testergebnisse analysiert und visualisiert.

---

### **1. Fehlertoleranz-Test: Worker-Ausfall**

Der erste Test untersucht die **Fehlertoleranz einer Spark-Anwendung**, wenn während der Laufzeit **Worker-Nodes ausfallen**. Ziel dieses Tests ist es, die Auswirkungen solcher plötzlichen Ausfälle zu analysieren und zu verstehen, wie Spark mit diesen umgeht.

#### **Fragestellung**
- **Was passiert, wenn Worker-Nodes in einer laufenden Spark-Anwendung ausfallen?**
- **Kann die Anwendung den Betrieb stabil fortsetzen?**

#### **Durchführung**
In einem Experiment wurden **zwei Worker-Nodes deaktiviert**, während eine Spark-Anwendung bereits seit **1 Minute** mit **1000 Dateien** lief. Die Deaktivierung wurde simuliert, indem die entsprechenden **cmd-Fenster der Worker geschlossen** wurden.

#### **Ergebnisse und Beobachtungen**

1. **Normale Verarbeitung (alle Worker aktiv)**  
   Zu Beginn des Tests waren alle **4 Worker aktiv**. Jeder Worker bearbeitete dabei **2 Tasks parallel**. Dies entsprach der ursprünglichen Konfiguration mit **2 CPU-Kernen pro Worker**.

   <img src="../data/images/workerall.png" alt="Alle Worker aktiv" width="800">

   *Abbildung: Taskbearbeitung bei 4 aktiven Workern (je 2 parallele Tasks pro Worker)*

2. **Ausfall von zwei Workern**  
   Nach der Deaktivierung von zwei Workern wurden die laufenden Tasks der betroffenen Worker **fehlgeschlagen**. Diese fehlerhaften Tasks wurden jedoch automatisch von den verbleibenden Workern neu übernommen und erfolgreich beendet.

   <img src="../data/images/workerloss1.png" alt="Worker-Ausfall sichtbar" width="600">

   *Abbildung: Ausfall von zwei Workern sichtbar in der Spark-UI*

   <img src="../data/images/workerloss2.png" alt="Fehlgeschlagene Tasks übernommen" width="700">

   *Abbildung: Neuverteilung der fehlgeschlagenen Tasks auf die verbleibenden Worker*

3. **Verarbeitung mit verbleibenden Workern**  
   Nach dem Ausfall wurden **alle weiteren Tasks** von den **verbleibenden zwei Workern** bearbeitet. 

   <img src="../data/images/workerleft.png" alt="Bearbeitung durch verbleibende Worker" width="800">

   *Abbildung: Taskbearbeitung nach dem Ausfall - übrig gebliebene Worker übernehmen Workload von den ausgefallenen Workern*

#### **Fazit**
Der Test zeigt, dass die Spark-Anwendung nach einem **Worker-Ausfall stabil weiterläuft**, indem die fehlgeschlagenen Tasks auf die verbleibenden Worker verteilt werden. Allerdings verlängerte sich dadurch die Durchlaufzeit der Ausführung, da die Anzahl an parallelen Tasks pro Worker immernoch bei 2 Tasks blieb.

---

### **2. Fehlertoleranz-Test: Netzwerk-Ausfall**

#### **Fragestellungen:**  
- Wie reagiert Spark, wenn die Verbindung zwischen einem Worker-Node und dem Master während der Laufzeit unterbrochen wird?  
- Werden die betroffenen Tasks automatisch auf andere Worker umverteilt, sodass die Verarbeitung stabil fortgesetzt wird?  

#### **Durchführung:**  
In diesem Experiment wurde die Verbindung zwischen einem **Worker-Node** und dem **Master-Node** gezielt getrennt, während eine Spark-Anwendung bereits seit **3 Minuten mit 5000 Dateien** lief. Die Simulation des Netzwerkausfalls erfolgte über die gezielte **Beendigung des Worker-Prozesses** in der Windows-Kommandozeile.  

Das Experiment wurde mit dem **gleichen Standalone-Spark-Cluster** durchgeführt, bestehend aus:  
- **1 Master-Node**  
- **4 Worker-Nodes**  
- **Identischen Hardware- und Softwarekonfigurationen wie im ersten Test**  

Um den Netzwerkausfall zu simulieren, wurde zunächst die **Portnummer des Workers** identifiziert, um darüber die **zugehörige Prozess-ID (PID)** zu bestimmen. Mithilfe der PID wurde dann der **Worker-Prozess während der Laufzeit gezielt beendet**, um eine Netzwerkunterbrechung zwischen Master und Worker nachzubilden.  

In [ ]:
taskkill /PID 7436 /F
ERFOLGREICH: Der Prozess mit PID 7436 wurde beendet.

#### **Ergebnisse und Beobachtungen**

Die Ergebnisse dieses Experiments zeigen eine starke Übereinstimmung mit den Beobachtungen aus dem ersten Test. Auch hier wurde durch die **Simulation eines Netzwerkausfalls** gezielt ein **aktiver Worker während der Laufzeit entfernt**, um die Reaktion von Spark auf einen plötzlichen Verbindungsverlust zu analysieren.

Wie bereits im vorherigen Experiment konnten vier Phasen identifiziert werden:

1. **Normale Verarbeitung (alle Worker aktiv)**  
   Zu Beginn des Tests waren alle **4 Worker aktiv**, und jeder bearbeitete **2 Tasks parallel**. Die Last war gleichmäßig verteilt, und alle Worker arbeiteten effizient zusammen.

2. **Netzwerktrennung zwischen Master und einem Worker**  
   Durch die simulierte Netzwerktrennung wurde die Kommunikation zwischen dem **Master und einem Worker** unterbrochen. Der Master konnte den Worker nicht mehr erreichen und markierte ihn als **DEAD**.

   <img src="../data/images/networkloss.png" alt="Fehlgeschlagene Tasks übernommen" width="700">

   *Abbildung: Ausfall eines Workers sichtbar in der Master-UI*  
    
Alle laufenden Tasks auf diesem Worker sind **fehlgeschlagen**. Diese fehlgeschlagenen Tasks wurden automatisch von den verbleibenden Workern **übernommen und erfolgreich abgeschlossen**.  

   <img src="../data/images/networkloss1.png" alt="Fehlgeschlagene Tasks übernommen" width="600">
 
   *Abbildung: Ausfall eines Workers sichtbar in der Spark-UI*  

4. **Verarbeitung mit verbleibenden Workern**  
   Nach dem Ausfall wurden **alle verbleibenden Tasks** von den verbliebenen **drei Workern** verarbeitet. Spark hat die Workload ohne Unterbrechung auf die verfügbaren Ressourcen umverteilt.

#### **Fazit**  
Dieses Experiment bestätigt erneut die **Fehlertoleranz von Apache Spark** gegenüber plötzlichen Worker-Ausfällen. Trotz der Netzwerktrennung kam es **weder zu einem Abbruch der Anwendung noch zu einem Datenverlust**.  
- Spark hat den ausgefallenen Worker korrekt erkannt und die fehlgeschlagenen Tasks **automatisch auf andere Worker verteilt**.  
- Die Verarbeitung lief **stabil weiter**, ohne dass ein manueller Eingriff erforderlich war.  

---


### **3. Fehlertoleranz-Test: Speicherbeschränkung eines Workers**

#### **Fragestellung:**  
- Wie reagiert Spark, wenn einem Worker im Cluster **deutlich weniger RAM** zur Verfügung steht als den anderen?  
- Wird dieser Worker weiterhin für die Verarbeitung genutzt oder wird er mit der Zeit *abgehängt* werden?  

#### **Durchführung:**  
In diesem Experiment wurde einem der vier Worker bewusst **ein deutlich geringerer Speicher zugewiesen** als den anderen. Während drei Worker mit **4 GB RAM** konfiguriert wurden, erhielt der vierte Worker lediglich **512 MB RAM**.

In [ ]:
Worker 1: spark-class org.apache.spark.deploy.worker.Worker --cores 2 --memory 512M spark://localhost:7077
Worker 2: spark-class org.apache.spark.deploy.worker.Worker --cores 2 --memory 4G spark://localhost:7077
Worker 3: spark-class org.apache.spark.deploy.worker.Worker --cores 2 --memory 4G spark://localhost:7077
Worker 4: spark-class org.apache.spark.deploy.worker.Worker --cores 2 --memory 4G spark://localhost:7077

  #### **Beobachtungen:**  
- Bereits vor der Ausführung der Spark-Anwendung wurde in der **Master UI** sichtbar, dass die konfigureierten Ressourcen des Workers (512 MB RAM und 2 Kerne)  als *"Not Used"* markiert wurde.

   <img src="../data/images/notusedworker.png" alt="Fehlgeschlagene Tasks übernommen" width="700">
   
    *trotz verfügbarer Ressourcen des Workers gibt Spark hier `0 Used` an (letzte Zeile)*

- Während der Laufzeit der Anwendung führte **Spark keine Tasks auf diesem Worker aus**.  
- In der **Spark UI** war ersichtlich, dass nur die drei Worker mit **4 GB RAM** aktiv Tasks bearbeiteten. Der vierte Worker wurde ignoriert.

   <img src="../data/images/workernotused.png" alt="Fehlgeschlagene Tasks übernommen" width="600">
   
    *3 Worker arbeiten an den Tasks - Worker 4 hat nicht mal eine Lane*

- Wegen dieser unerwarteten Beobachtung, wurden nach Begründungen für dieses Verhalten des Systems gesucht.

**--> Folgendes ist aufgefallen:**
- Während bei dem Worker mit nur **512 MB RAM "not Used"** vermerkt war, stand bei den Workern mit 4 GB RAM nicht etwa *"4 GB Used"* sondern *"1024 MB Used"*. 
- Unter `Executor Memory - Default Resource Profile` wird außerdem der Wert **1024.0 MiB** angezeigt, was darauf hinweist, dass Spark standardmäßig mindestens **1 GB RAM pro Executor benötigt**.  
- Da der Worker mit **512 MB RAM** diese Anforderung nicht erfüllte, wurde er von Spark **komplett ignoriert**.  

#### **Fazit:**  
Dieses Experiment zeigt, dass Spark **keine Worker verwendet, die die Mindestanforderung an `spark.executor.memory` nicht erfüllen**.  
- Da die Standard-Einstellung für `spark.executor.memory` mindestens **1 GB (1024 MiB)** beträgt, wurde der Worker mit **nur 512 MB nicht berücksichtigt**.  
- Um den Test erfolgreich durchführen zu können, muss der **Speicherbedarf pro Executor reduziert** werden. Dies kann durch die Konfiguration von `spark.executor.memory` erfolgen.

**-->** ***Was wenn spark.executor.memory konfiguriert wurde - Wie reagiert Spark bei nun ungleichmäßier Belastung von Nodes?***

Hätte das Experiment funktioniert wären sehr wahrscheinliche Ähnliche Mechanismen wie bei den ersten beiden Tests zu beobachten. Denn auch hier ist Spark fehlertolerant, indem er ungleichmäßige Last durch interne Mechanismen ausgleicht.

**Task-Neuverteilung:** Wenn ein überlasteter Node langsamer wird oder ausfällt, übernimmt Spark automatisch die betroffenen Tasks auf andere verfügbare Nodes.  
**Automatische Wiederherstellung:** Selbst bei stark ungleicher Last bleibt die Verarbeitung stabil, da Spark blockierte oder ausgefallene Tasks neu plant.  

- Spark bleibt also auch bei ungleichmäßiger Belastung fehlertolerant!  


---

### **Weitere potenzielle Fehlerszenarien und deren Auswirkungen**

Neben den getesteten Szenarien gibt es weitere mögliche Fehlerfälle, die in einer verteilten Spark-Anwendung auftreten können. Die Reaktion des Systems hängt stark von der Implementierung und den Konfigurationen ab.

- **Master-Ausfall**  
   - Falls der Spark-Master während der Laufzeit ausfällt, können **keine neuen Tasks mehr geplant** werden.  
   - Bereits laufende Tasks auf den Workern werden weiter ausgeführt, aber neue Stages können nicht gestartet werden.  

- **Out-of-Memory-Probleme bei Workern**  
   - Falls ein Worker den Speicher überschreitet, schlägt der betroffene Task fehl, und Spark versucht ihn auf einem anderen Worker auszuführen.  
   - Unser Code prüft bereits **Zeilenanzahlen nach Joins**, um große Datenverluste früh zu erkennen.  
   - Eine präzisere Speichersteuerung durch bspw. **persist()** könnte helfen.  

- **Fehlerhafte Eingabedateien**  
   - Falls beschädigte oder unvollständige CSV- oder Pickle-Dateien vorliegen, kann dies zu unerwarteten Fehlern führen.  
   - Unser Code erkennt solche Probleme teilweise durch **Zeilenüberprüfung nach Transformationen** und durch einen **try-except-Block für fehlerhafte Datei-Reads**
   - Weitere Maßnahmen könnten **Validierungschecks auf Spaltennamen und Datentypen** sein.  

- **Langsame oder nicht erreichbare externe Datenquellen**  
   - Falls eine externe Datenquelle nicht erreichbar ist, kann die Anwendung blockieren oder fehlschlagen.  
   - Unser Code würde dies aktuell nicht direkt abfangen, aber **Retry-Mechanismen oder alternative Datenquellen** könnten die Fehlertoleranz verbessern.  
